# Análise Eleitoral

### 0) Configuração dos bancos a serem utilizados

In [1]:
import pandas as pd
import tse

In [2]:
CANDIDATO = 'GUILHERME CASTRO BOULOS'
ESTADO = 'sp' # lowercase!

In [3]:
database_1 = f'databases/{ESTADO}_2024.sql'
database_2 = 'databases/eleicoes_2024.sql'

In [4]:
analise = tse.TseAnalysis(
    database={
        'path': database_1,
        'alias': f'{ESTADO}'
    })

In [5]:
analise.attach_database(
    attach= [
        {
            'path': database_2,
            'alias': 'geral'
        }
    ])

Bancos carregados:
0) main
-path: d:\projects\tse-data\databases\rj_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
2) rj
-path: d:\projects\tse-data\databases\rj_2024.sql
--tables: 
-- resultados
-- eleitorado
-----
3) geral
-path: d:\projects\tse-data\databases\eleicoes_2024.sql
--tables: 
-- locais
-----


In [6]:
analise.create_index(
    table='resultados',
    columns=['NM_VOTAVEL', 'NR_VOTAVEL', 'NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO', 'SG_UF']
)
analise.create_index(
    table='eleitorado',
    columns=['NR_LOCAL_VOTACAO', 'NR_ZONA', 'NR_SECAO', 'CD_MUNICIPIO', 'SG_UF']
)

'Indices criados com sucesso'

---

### 1) Criação de tabela com perfil de eleitorado por seção

In [7]:
# a função generate_stats_query foi escrita para gerar uma query específica para a tabela {ESTADO}. eleitorado
# ela analisa os valores distintos de cada coluna argumento e produz uma query que soma a quantidade de aparições para cada caso
q = analise.generate_stats_query(
    alias=f'{ESTADO}',
    table='eleitorado',
    cols=[
        'DS_GRAU_ESCOLARIDADE',
        'DS_RACA_COR',
        'DS_FAIXA_ETARIA',
        'DS_GENERO'
    ]
)

In [8]:
eleitorado_secao = pd.read_sql(q, analise.conn)

In [9]:
eleitorado_secao.head(2) # este DataFrame expressa a quantidade de eleitores por seçao divido em categorias

,municipio,local,zona,secao,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_lê_e_escreve,ds_grau_escolaridade_ensino_médio_completo,...,ds_faixa_etaria_85_a_89_anos,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado
0,58009,1015,255,23,61,23,114,37,42,91,...,10,4,0,1,7,0,0,175,211,0
1,58009,1015,255,24,79,29,99,26,48,83,...,2,0,1,0,10,1,0,183,203,0


In [10]:
q_2 = f"""
    SELECT
        CD_MUNICIPIO as municipio,
        NR_LOCAL_VOTACAO as local,
        NR_ZONA as zona,
        NR_SECAO as secao,
        QT_ELEITOR_SECAO as eleitores_totais
    FROM
        geral.locais
    WHERE
        SG_UF = '{ESTADO.upper()}'
"""

In [11]:
eleitores_secao = pd.read_sql(q_2, analise.conn)

In [12]:
eleitores_secao.head(2) # complementa o DataFrame anterior com a quantidade total de eleitores na seção

,municipio,local,zona,secao,eleitores_totais
0,60011,1252,185,136,406
1,60011,1341,14,164,446


In [13]:
eleitores_secao.loc[(eleitores_secao['secao'] == 284) & (eleitores_secao['zona'] == 233)]

,municipio,local,zona,secao,eleitores_totais
28889,60011,1368,233,284,64


In [14]:
eleitorado_secao = pd.merge(
    eleitorado_secao,
    eleitores_secao,
    how='inner',
    on=[
        'municipio',
        'local',
        'zona',
        'secao'
    ]
)

In [15]:
eleitorado_secao.head(2) # resultado obtido

,municipio,local,zona,secao,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_lê_e_escreve,ds_grau_escolaridade_ensino_médio_completo,...,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado,eleitores_totais
0,58009,1015,255,23,61,23,114,37,42,91,...,4,0,1,7,0,0,175,211,0,386
1,58009,1015,255,24,79,29,99,26,48,83,...,0,1,0,10,1,0,183,203,0,386


In [16]:
eleitorado_secao.loc[(eleitorado_secao['secao'] == 284) & (eleitorado_secao['zona'] == 233)]

,municipio,local,zona,secao,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,ds_grau_escolaridade_ensino_fundamental_completo,ds_grau_escolaridade_lê_e_escreve,ds_grau_escolaridade_ensino_médio_completo,...,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado,eleitores_totais
35202,60011,1368,233,284,10,2,6,3,1,36,...,0,0,0,0,1,0,47,17,0,64


---

### 2) Cria tabela com apuração dos resultados por seção de determinado candidato

In [17]:
query = f"""
    SELECT
        NM_VOTAVEL as candidato,
        CD_MUNICIPIO as municipio,
        NR_LOCAL_VOTACAO as local,
        NR_ZONA as zona,
        NR_SECAO as secao,
        NR_TURNO as turno,
        QT_VOTOS as votos_recebidos
    FROM
        {ESTADO}.resultados
    WHERE
        NM_VOTAVEL LIKE '{'%' + CANDIDATO.replace(' ', '%') + '%'}'
"""

In [18]:
resultado_secao = pd.read_sql(query, analise.conn)

In [19]:
resultado_secao.head(2)

,candidato,municipio,local,zona,secao,turno,votos_recebidos
0,ALEXANDRE RAMAGEM RODRIGUES,60011,1031,182,181,1,116
1,ALEXANDRE RAMAGEM RODRIGUES,60011,1147,167,101,1,50


In [20]:
resultado_secao.loc[(resultado_secao['secao'] == 284) & (resultado_secao['zona'] == 233)]

,candidato,municipio,local,zona,secao,turno,votos_recebidos
4815,ALEXANDRE RAMAGEM RODRIGUES,60011,1368,233,284,1,128


In [21]:
resultado_perfil = pd.merge(
    resultado_secao,
    eleitorado_secao,
    how='inner',
    on=['municipio', 'local', 'zona', 'secao']
)

In [22]:
resultado_perfil.head(2)

,candidato,municipio,local,zona,secao,turno,votos_recebidos,ds_grau_escolaridade_ensino_médio_incompleto,ds_grau_escolaridade_superior_completo,ds_grau_escolaridade_ensino_fundamental_incompleto,...,ds_faixa_etaria_90_a_94_anos,ds_faixa_etaria_100_anos_ou_mais,ds_faixa_etaria_95_a_99_anos,ds_faixa_etaria_17_anos,ds_faixa_etaria_16_anos,ds_faixa_etaria_inválida,ds_genero_feminino,ds_genero_masculino,ds_genero_não_informado,eleitores_totais
0,ALEXANDRE RAMAGEM RODRIGUES,60011,1031,182,181,1,116,174,21,89,...,0,0,0,0,0,0,246,235,0,481
1,ALEXANDRE RAMAGEM RODRIGUES,60011,1147,167,101,1,50,55,15,64,...,5,9,11,1,0,0,156,126,2,284


In [23]:
resultado_perfil['percentual_votos'] = resultado_perfil['votos_recebidos']/resultado_perfil['eleitores_totais']

In [24]:
nome = resultado_perfil['candidato'][0]
resultado_perfil.to_csv(f'resultados_secao_absolutos/{nome.replace(' ', '_')}.csv')